In [1]:
%pylab inline
import numpy as np
import nibabel as nib
from scipy.stats import pearsonr
from sklearn.decomposition import FastICA

Populating the interactive namespace from numpy and matplotlib


C:\Users\Lixuan\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def datafunction(img, atlas, region, n_components, filename):
    #img and atlas must be inputed as strings.
    #Img refers to data sets to be visualized. 
    #atlas refers the brain whole brain data
    #region refer specific region that is to be extracted and analyzed. Must be an intiger
    # n_comonents refers to number of components for the FastICA to use. Must be intiger
    # filename refers to what the name of the output file. Must be a string. 
    
    img = nib.load(img)
    atlas = nib.load(atlas)
    
    #transfers data to matrix
    img_data = np.asarray(img._dataobj)
    atlas_data = np.asarray(atlas._dataobj)
    
    #Region refers to region number in ITK
    region_mask = (atlas_data == region) #extracts region data
    region_indices = np.nonzero(region_mask)
    
    ica = FastICA(n_components) 
    
    X = np.transpose(img_data[region_indices])
    icomps = ica.fit_transform(X)

    output = []
    for i, c in enumerate(icomps.T):
        output.append(np.zeros(X.shape[1]))
        for v in range (X.shape[1]):
            output[i][v] = pearsonr(c, X[:, v])[0]
    
    output_volume = np.zeros(atlas_data.shape)
    for i, (x, y, z) in enumerate(np.transpose(region_indices)):
        output_volume[x, y, z] = output[0][i]
        
    
    #Saves to file
    nii = nib.Nifti1Image(output_volume, atlas.affine)
    nii.to_filename(filename)
    print('Output saved to ' + filename)
    
    

In [3]:
#Example function call:

datafunction('OneDrive_1_11-16-2018/Data/sub-F01_task-rs_bold.nii.gz',
             'OneDrive_1_11-16-2018/Atlas/Rat_Paxinos_400um_bilat_180823_atlas.nii.gz',
             27, 5, '1_comp.nii.gz')